In [1]:
import sys 
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import  accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('E:\\备份\\训练数据汇总(8W)\\2019_4_15.csv')
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

name = ['PHONE_VERSION', 'VIDEO_CLARITY']
columns = data.columns.values.tolist()
name_1 = [name for index, name in enumerate(columns) if name != name[0] and name != name[1]]

scaler = MinMaxScaler()
X1 = data[name_1[0:16]]
X1_data = scaler.fit_transform(X1)
Y = data[name_1[16:21]]
X2 = data[name[0]]
X3 = data[name[1]]

X2_data = OneHotEncoder().fit_transform(X2.values.reshape(-1, 1)).todense().getA()
X3_data = OneHotEncoder().fit_transform(X3.values.reshape(-1, 1)).todense().getA()

X_data = np.hstack((X1_data, X2_data, X3_data))

Y1 = data[[name_1[16]]]
Y2 = data[[name_1[17]]]
Y3 = data[[name_1[18]]]
Y4 = data[[name_1[19]]]

X1_train, X1_test, y1_train, y1_test = train_test_split(X_data, Y1, test_size=0.25, random_state = 33)
X2_train, X2_test, y2_train, y2_test = train_test_split(X_data, Y2, test_size=0.25, random_state = 33)
X3_train, X3_test, y3_train, y3_test = train_test_split(X_data, Y3, test_size=0.25, random_state = 33)
X4_train, X4_test, y4_train, y4_test = train_test_split(X_data, Y4, test_size=0.25, random_state = 33)

y1_train = OneHotEncoder().fit_transform(y1_train.values.reshape(-1, 1)).todense().getA()
y1_test_code = OneHotEncoder().fit_transform(y1_test.values.reshape(-1, 1)).todense().getA()


In [2]:
def generatebatch(X,Y,n_examples, batch_size): 
    for batch_i in range(n_examples // batch_size): 
        start = batch_i * batch_size 
        end = start + batch_size 
        batch_xs = X[start:end] 
        batch_ys = Y[start:end] 
        yield batch_xs, batch_ys 

In [3]:
import tensorflow as tf

def RNN(X,weights,biases):
    # hidden layer for input
    X = tf.reshape(X, [-1, n_inputs])
    X_in = tf.matmul(X, weights['in']) + biases['in']
    X_in = tf.reshape(X_in, [-1,n_steps, n_hidden_units])
    
    # cell
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units, forget_bias=1.0, state_is_tuple=True)
    _init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)
    outputs,states = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=_init_state, time_major=False)
    
    #hidden layer for output as the final results
    #results = tf.matmul(states[1], weights['out']) + biases['out']
    # or
    outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
    results = tf.matmul(outputs[-1], weights['out']) + biases['out']

    return results

# parameters init
l_r = 0.001
training_iters = 100
batch_size = 128

n_inputs = 7
n_steps = 4
n_hidden_units = 128
n_classes = 5

#define placeholder for input
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])

# define w and b
weights = {
    'in': tf.Variable(tf.random_normal([n_inputs,n_hidden_units])),
    'out': tf.Variable(tf.random_normal([n_hidden_units,n_classes]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1,shape=[n_hidden_units,])),
    'out': tf.Variable(tf.constant(0.1,shape=[n_classes,]))
}

pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
train_op = tf.train.AdamOptimizer(l_r).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

#init session
sess = tf.Session()
#init all variables
sess.run(tf.global_variables_initializer())
#start training

In [4]:
for i in range(training_iters):
    for batch_xs,batch_ys in generatebatch(X1_train, y1_train, y1_train.shape[0], batch_size):
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run(train_op,feed_dict={x: batch_xs, y: batch_ys})
    if i % 100 == 0:
        print(sess.run(accuracy,feed_dict={x: batch_xs, y: batch_ys}))

0.765625


In [8]:
test_data = X1_test.reshape([-1, n_steps, n_inputs])
test_label = y1_test_code

In [11]:
for batch_xs,batch_ys in generatebatch(X1_test, y1_test, y1_test.shape[0], batch_size):
    batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
    y_pre = sess.run(pred, feed_dict={x:batch_xs})